
# Trabajo Práctico 2

Dentro de la carpeta de `data/` vamos a poder encontrar dos datasets:

- Pokemon (Problema de clasificacion Multivariada)
- Challenger Space Shuttle O-Ring Data (Problema de Regresion multivariada)


# Importar librerías

Importar aquellas librerías que serán utilizadas en el trabajo.

# Cargar datos
Cargar los datos de entrenamiento.

# Análisis exploratorio básico y preprocesamiento de los datos

Análisis de los datos para conocer los mismos, ver datos faltantes, decidir cómo tratarlos, ver distribuciones, relaciones, etc. Procesar los datos centrándolos, reescalando, codificando, reduciendo dimensiones, etc. según considere necesario. 

# Modelos y evaluación
Probar diferentes modelos para predecir la variable objetivo. Calcular las métricas que considere relevantes. Comentar los resultados. 